In [11]:
import pandas as pd
import os
import json
import numpy as np
from Preprocessor import preprocessor
import keras.layers as L
import keras.activations as A
import keras.optimizers as O
import keras.losses as Loss
from keras.models import Model

In [30]:
P=preprocessor(r'C:/Users/MuhammadUsman/PycharmProjects/internship.ai')
config={'data_directory':r'C:\Users\MuhammadUsman\PycharmProjects\internship.ai\Output','file_batch_size':3,'format_':'h5','ex_batch_size':100,'file_examples':1000,'obsnet_col':4,'Preprocessor':P}

In [31]:
import os
import numpy as np
import keras
import glob
class Generator(keras.utils.Sequence):
    def __init__(self, config):
        self.config = config
        self.DataDirectory = self.config['data_directory']
        self.DataFiles = np.array(glob.glob(self.DataDirectory+'/*'+self.config['format_']))
        self.Indexes_files = np.array(list(range(len(self.DataFiles))))
        self.P=self.config['Preprocessor']
        self.Data = []
        self.i_e = 0
        self.i_f = 0

    def fill_buffer(self):
        files_batch_size = self.config['file_batch_size']
        files=self.DataFiles[self.Indexes_files[self.i_f:self.i_f+files_batch_size]]
        self.Data=self.load_files(files, self.config['format_'])
        self.i_f += files_batch_size

    def load_files(self, files, format_):
        data_list = []
        if format_ == 'csv':
            for i in files:
                if i[-4:] == '.csv':
                    array = pd.read_csv(os.path.join(self.DataDirectory, i))
                    array=P.process_batch(array)
                    self.obsnet_col=array.columns=='ob_net'
                    self.action_col=array.columns=='action'
                    data_list.extend(array.values)
        elif format_ == 'h5':
            for i in files:
                if i[-3:] == '.h5':
                    array = pd.read_hdf(os.path.join(self.DataDirectory, i))
                    array=P.process_batch(array)
                    self.obsnet_col=array.columns=='ob_net'
                    self.action_col=array.columns=='action'
                    data_list.extend(array.values)
        elif format_ == 'npy':
            for i in files:
                if i[-4:] == '.csv':
                    array = np.load(os.path.join(self.DataDirectory, i))
                    data_list.extend(array)
        return np.array(data_list)
    def load_data(self):
        self.examples_batch_size = self.config['ex_batch_size']
        if self.i_f >= len(self.Indexes_files):
            np.random.shuffle(self.Indexes_files)
            self.i_f = 0
        if self.i_e >= len(self.Data):
            self.fill_buffer()
            self.indexes_examples = np.arange(0,len(self.Data),dtype= np.int32)
            np.random.shuffle(self.indexes_examples)
            self.i_e = 0
        res=self.Data[self.indexes_examples[self.i_e:self.i_e+self.examples_batch_size]]
        self.i_e += self.examples_batch_size
        return res
    def __len__(self):
        return (len(self.DataFiles)*self.config['file_examples'])//self.config['ex_batch_size']
    def on_epoch_end(self):
        pass
    def __getitem__(self, idx):
        data = self.load_data()
        data1 = data[:,(~self.obsnet_col) & (~self.action_col)]
        obsnet = data[:,self.obsnet_col]
        action=data[:,self.action_col]
        list_obsnet=[]
        for i in obsnet:
            list_obsnet.append(np.array(i[0]).flatten())
        obsnet=np.array(list_obsnet)
        return np.c_[data1, obsnet],keras.utils.to_categorical(action,5)
G=Generator(config=config)

In [32]:
class FC_Model():
    def __init__(self,config):
        self.Neurons=config['Hidden']
        self.activation=config['Activation']
        self.output=config['Output']
        self.input_dim=config['Input_shape']
        self.optimizer=config['Optimizer']
        self.model=self.make_model()
        self.loss()
    def make_model(self):
        x=L.Input(shape=(self.input_dim,))
        inp=x
        for i in self.Neurons:
            x=L.Dense(i,activation=self.activation)(x)
        x=L.Dense(self.output,activation='softmax')(x)
        return Model(inp,x)
    def loss(self):
        self.model.compile(loss=Loss.categorical_crossentropy, optimizer=self.optimizer)
    def fit(self,generator):
        self.model.fit_generator(generator= generator, 
                    steps_per_epoch  = len(generator), 
                    epochs           = 10, 
                    verbose          = 1,
#                     validation_data  = valid_batch,
#                     validation_steps = len(valid_batch),
                    max_queue_size   = 3)        
n_config={'Hidden':[10,10],'Activation':'relu','Output':5,'Optimizer':O.Adam(lr=0.5e-5),'Input_shape':472 }
model=FC_Model(n_config)

In [33]:
model.fit(generator=G)

Epoch 1/10


C:\ProgramData\Anaconda2\envs\py3\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.19.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


100/100 [==============================] - 6s 55ms/step - loss: 16.0810 1s - l
Epoch 2/10
100/100 [==============================] - 4s 43ms/step - loss: 16.0810
Epoch 3/10
100/100 [==============================] - 4s 44ms/step - loss: 16.0810
Epoch 4/10
100/100 [==============================] - 5s 51ms/step - loss: 16.0810
Epoch 5/10
100/100 [==============================] - 5s 46ms/step - loss: 16.0810
Epoch 6/10
100/100 [==============================] - 5s 51ms/step - loss: 16.0810
Epoch 7/10
100/100 [==============================] - 4s 44ms/step - loss: 16.0810
Epoch 8/10
100/100 [==============================] - 5s 47ms/step - loss: 16.0810
Epoch 9/10
100/100 [==============================] - 4s 44ms/step - loss: 16.0810
Epoch 10/10
100/100 [==============================] - 5s 45ms/step - loss: 16.0810
